In [8]:
#必要なライブラリ、定数
import numpy as np
import random
import time
import copy
random.seed(time.time())
#below is the info of basemat
N=16 #log_2(N)==int <-- requirement
K=8
design_SNR=1

itr_num=np.log2(N)
itr_num=itr_num.astype(int)

G2=np.array([[1,0],[1,1]],dtype=np.int)

In [9]:
def tensordot(A):
  tmp0=np.zeros((A.shape[0],A.shape[1]),dtype=np.int)
  tmp1=np.append(A,tmp0,axis=1)
  #print(tmp1)
  tmp2=np.append(A,A,axis=1)
  #print(tmp2)
  tmp3=np.append(tmp1,tmp2,axis=0)
  #print(tmp3)
  return tmp3

Gres=G2
for i in range(itr_num-1):
  #print(i)
  Gres=tensordot(Gres)
print(Gres)

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0]
 [1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


In [10]:
def Bhattacharyya_bounds(N,K,EdB):
  E = 10 ** (EdB / 10)
  itr_num=np.log2(N)
  itr_num=itr_num.astype(int)
  z=np.zeros(N)
  z[0]=np.exp(-E)
  for j in range(itr_num):
    tmp=2**(j)
    for t in range(tmp):
      T=z[t]
      z[t]=2*T-T**2
      z[tmp+t]=T**2
  res=indices_of_elements(z,N-K)
  return res

def indices_of_elements(v,l):
  tmp=np.argsort(v)[::-1]
  np.savetxt("frozen_bit.txt",tmp)
  res=tmp[0:l]
  return res

In [11]:
def generate_codeword(N,Gres):
  S=np.zeros(N)
  for i in range(N):
    S[i]=random.randrange(0,2)
    np.put(S,frozen_bit,0)
  U=(S@Gres)%2
  #print(U.shape)
  return S,U

In [12]:
def AWGN_channel(EbNodB,codeword):
  #modulation-channel
  # 送信側アンテナ数
  M =1
  # 受信側アンテナ数
  N = 1
  # 送信ビット列
  TX_bit =copy.deepcopy(codeword)
  # 送信ビット数
  bit_num =codeword.shape[1]

  # Additive Gaussian White Noiseの生成する際のパラメータ設定
  EbNo = 10 ** (EbNodB / 10)
  No=1/EbNo #Eb=1(fixed)

  # 0 -> 1, 1 -> -1としてBPSK変調
  TX_BPSK = TX_bit
  TX_BPSK[TX_bit==1]=-1
  TX_BPSK[TX_bit==0]=1

  # AWGN雑音の生成
  noise = np.random.normal(0, np.sqrt(No / 2), (M, bit_num)) \
          #+ 1j * np.random.normal(0, np.sqrt(No / 2), (M, bit_num))

  # AWGN通信路 = 送信シンボル間干渉が生じないような通信路で送信
  RX_BPSK = TX_BPSK + noise

  # 以下のprint関数の出力を表示すると、Noとほぼ一致するはず
  #print(np.dot(noise[0, :], np.conj(noise[0, :]))/bit_num)
  
  return RX_BPSK

#output "RX_BPSK"(2D(M,len(codeword)))-array


In [13]:
#SC復号

#node operation
def SC_decoding(a):
  global t
  #interior node operation
  if a.shape[0]==1:
    #frozen_bit or not
    if np.any(frozen_bit==t):
      tmp=np.zeros(1)
    elif a>=0:
      tmp=np.zeros(1)
    elif a<0:
      tmp=np.ones(1)
    EST_codeword[t]=tmp
    #print(t)
    t+=1
    #if t>=N:
      #exit()
    return tmp

  #step1 left input a output u1_hat
  tmp1=np.split(a,2)
  tmp10=a.reshape([2,a.shape[0]//2])
  f_half_a=np.sign(tmp1[0])*np.sign(tmp1[1])*np.amin(np.abs(tmp10),axis=0)
  u1=SC_decoding(f_half_a)

  #step2 right input a,u1_hat output u2_hat 
  tmp2=np.split(a,2)
  g_half_a=tmp2[1]+(1-2*u1)*tmp2[0] 
  u2=SC_decoding(g_half_a)
  
  #step3 up input u1,u2 output a_hat
  res=np.concatenate([(u1+u2)%2,u2])
  return res

In [14]:
EbNodB=0
#frozen bit を定める

frozen_bit=Bhattacharyya_bounds(16,8,-5)

sourcecode,codeword_1D=generate_codeword(N,Gres) #1D-array codeword
codeword=codeword_1D[...,np.newaxis] 
codeword=np.transpose(codeword) #2D-array codeword
print(frozen_bit)
print(sourcecode,codeword_1D)
RX_BPSK=AWGN_channel(EbNodB,codeword)

#受信信号の1列ごとの受け渡し(数値をx軸のみに変更)
y=RX_BPSK[0,:].real #1D-array codeword
print(y)

#LLR
EbNo = 10 ** (EbNodB / 10)
No=1/EbNo
Lc=4*y/No
print(Lc)
t=0
EST_codeword=np.zeros(N)
SC_decoding(Lc)
#t=0
#EST_codeword=np.zeros(N)
#SC_decoding(y)

#EST_codeword=np.sign(y)
#EST_codeword[EST_codeword==1]=0
#EST_codeword[EST_codeword==-1]=1
#EST_codeword=(EST_codeword@Gres)%2
#print(sourcecode)
#print(EST_codeword)
#print(np.sum(sourcecode!=EST_codeword))
#print(sourcecode)
#print(EST_codeword)
#print(np.any(frozen_bit==500))

#input "PC_MATRIX"(2D(M,:)-array)

[ 0  8  4  2  1 12 10  6]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.] [0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1.]
[-0.41800808  0.76559817 -0.49654886 -1.4183963   0.68090208  0.92109585
 -1.02077393 -0.54000173  0.44218489 -1.69908039  2.8304755  -0.0090189
  1.03918014 -0.02486647 -0.00965457 -1.69812198]
[-1.67203233  3.0623927  -1.98619546 -5.67358518  2.72360834  3.68438338
 -4.08309571 -2.16000692  1.76873957 -6.79632157 11.32190201 -0.03607562
  4.15672055 -0.09946586 -0.03861829 -6.79248794]


array([0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.])